Copyright 2022 Abigail Harrison

Permission is hereby granted, free of charge, to any person obtaining a copy of this software 
and associated documentation files (the "Software"), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial 
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, 
INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE 
AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, 
DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, 
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
from Election_Simulation import *
from polling_backend import *

In [2]:
#Gathers Election data from Simulation_Input.txt to create a simulated election to audit
#In a real audit, upload the necessary files to the directory and do not run this code block
simulationData, margins = readInput()
numBallots, overvotes1, undervotes1, overvotes2, undervotes2, riskLimit, num, gamma = dataToValues(simulationData)
margin = margins[0][0]

if (os.path.exists("2020_CT_Election_Data.json")):
    #Imports JSON file with election population
    inputFile = open(os.path.join(sys.path[0], "2020_CT_Election_Data.json"), "r")
    jsonFile = json.load(inputFile)
    if jsonFile is None:
        raise SyntaxError("Something is wrong with the JSON file. Please check it and try again.")
else:
    jsonFile = None

E1 = Election(numBallots, margin, overvotes1, undervotes1, overvotes2, undervotes2, riskLimit, gamma, 1, jsonFile)
#Creates the ballot manifest and tabulation.
pollingSetup(E1)

Manifest created
Tabulation created


In [3]:
#Seed from the seed dice rolling ceremony. This is used to select ballots to audit
seed = 2368607141

In [4]:
#Reads in data from the tabulation and ballot manifest
numBallots, winnerBallots, runnerupBallots, tabulation_file, manifest_file = readFiles()
T = 1 #Test statistic for ballot polling risk limit
#Continues running rounds until risk limit is met
while True:
    #Calculate a sample size
    sampleSize = roundSample(numBallots, winnerBallots, runnerupBallots)
    #Select the ballots from the sample size
    ballotSelect(sampleSize, seed, manifest_file)
    #Enter the number of Winner ballots and Runnerup ballots observed in the sample
    sampledWinner, sampledRunnerup = roundInput()
    sampledWinner = int(sampledWinner)
    sampledRunnerup = int(sampledRunnerup)
    #Calculate the observed risk limit
    sw = winnerBallots/numBallots
    T = calculateRisk(sw, T, sampledWinner, sampledRunnerup)
    #Determine if more auditing is necessary
    if (T >= 1/E1.riskLimit):
        print("Audit complete. You may stop auditing. Observed risk limit = ", 1/T)
        break
    elif (1/T > 1): #How to actually get this value?
        raise ValueError("A full hand recount is necessary to determine the winner.")
    else:
        print("Another round is necessary. Observed risk limit for the round =", 1/T)
        winnerBallots = sampledWinner
        runnerupBallots = sampledRunnerup
        numBallots = sampleSize

Ballots to audit this round: 2527
Making ballot pull sheets now.
After sampling, there are 2488 ballots to pull. Check ballot_polling_pull_list folder for the list of ballots.


Please enter the number of winner ballots:  2000
Please enter the number of runnerup ballots:  488


Audit complete. You may stop auditing. Observed risk limit =  0.003254670732036218
